In [1]:
# Install faker if not exists
%pip install faker

import sqlite3
import os
import random
import pandas as pd
from datetime import datetime, timedelta
try:
    from faker import Faker
except ImportError:
    # Fallback if pip install didn't work in current kernel session immediately
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "faker"])
    from faker import Faker

# Initialize Faker
fake = Faker()

# Define database path
db_path = '../data/hotel_retention.db'

# Ensure directory exists
os.makedirs(os.path.dirname(db_path), exist_ok=True)

# Connect to database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Create table if not exists
create_table_sql = """
CREATE TABLE IF NOT EXISTS bookings (
    booking_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_id INTEGER NOT NULL,
    name TEXT NOT NULL,
    email TEXT NOT NULL,
    phone TEXT,
    room_type TEXT NOT NULL,
    booking_price REAL NOT NULL,
    booking_date TEXT NOT NULL,
    checkin_date TEXT NOT NULL,
    checkout_date TEXT NOT NULL,
    special_requests TEXT,
    total_stays INTEGER DEFAULT 1,
    previous_cancellations INTEGER DEFAULT 0,
    status TEXT DEFAULT 'Confirmed'
);
"""
cursor.execute(create_table_sql)
print("✅ Table 'bookings' check passed.")

# Clear existing data to avoid duplicates/ensure clean state (Optional - comment out if you want to append)
cursor.execute("DELETE FROM bookings")
print("🗑️  Cleared existing data.")

bookings = []

# 1. SPECIFIC TEST USER (Prakash Saini)
prakash_checkin = (datetime.now() + timedelta(days=2)).strftime("%Y-%m-%d")
prakash_checkout = (datetime.now() + timedelta(days=5)).strftime("%Y-%m-%d")

bookings.append((
    101, 
    "Prakash Saini", 
    "prakash.mali23@st.niituniversity.in", 
    "9876543210", 
    "Executive Suite", 
    1200.00, 
    "2024-01-10", 
    prakash_checkin, 
    prakash_checkout, 
    "Late checkout requested", 
    2, 
    1, 
    "Confirmed"
))

# 2. GENERATE 100 RANDOM CUSTOMERS
start_id = 102
room_types = ["Standard", "Deluxe", "Suite", "Presidential", "Executive Suite"]
statuses = ["Confirmed", "Pending", "Cancelled", "Checked Out"]

print("🎲 Generating 100 realistic customers...")

for i in range(100):
    cust_id = start_id + i
    name = fake.name()
    email = fake.email()
    phone = fake.phone_number()
    room = random.choice(room_types)
    price = round(random.uniform(100, 2000), 2)
    
    # Random dates
    booking_date = fake.date_between(start_date='-1y', end_date='today').strftime("%Y-%m-%d")
    checkin = fake.date_between(start_date='today', end_date='+2M').strftime("%Y-%m-%d")
    checkout = (datetime.strptime(checkin, "%Y-%m-%d") + timedelta(days=random.randint(1, 7))).strftime("%Y-%m-%d")
    
    requests = fake.sentence() if random.random() > 0.7 else None
    stays = random.randint(1, 20)
    cancels = random.randint(0, 5)
    status = random.choice(statuses)
    
    bookings.append((
        cust_id, name, email, phone, room, price, booking_date, checkin, checkout, requests, stays, cancels, status
    ))

sql = '''INSERT INTO bookings(customer_id, name, email, phone, room_type, booking_price, booking_date, checkin_date, checkout_date, special_requests, total_stays, previous_cancellations, status)
         VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?)'''

cursor.executemany(sql, bookings)
conn.commit()
print(f"✅ Inserted {len(bookings)} realistic bookings.")

# --- VERIFICATION ---

# 1. Check for Prakash Saini
print("\n--- Checking for Prakash Saini ---")
query_prakash = "SELECT * FROM bookings WHERE name LIKE '%Prakash Saini%'"
df_prakash = pd.read_sql_query(query_prakash, conn)

if not df_prakash.empty:
    print(f"✅ Found {len(df_prakash)} record(s) for 'Prakash Saini':")
    display(df_prakash)
else:
    print("❌ 'Prakash Saini' NOT found in the database.")

# 2. List ALL bookings
print("\n--- All Bookings in Database ---")
query_all = "SELECT * FROM bookings"
df_all = pd.read_sql_query(query_all, conn)

print(f"Total bookings found: {len(df_all)}")
display(df_all)

conn.close()

  Using cached faker-40.4.0-py3-none-any.whl.metadata (16 kB)
Using cached faker-40.4.0-py3-none-any.whl (2.0 MB)
Note: you may need to restart the kernel to use updated packages.
✅ Table 'bookings' check passed.
🗑️  Cleared existing data.
🎲 Generating 100 realistic customers...
✅ Inserted 101 realistic bookings.

--- Checking for Prakash Saini ---
✅ Found 1 record(s) for 'Prakash Saini':


,booking_id,customer_id,name,email,phone,room_type,booking_price,booking_date,checkin_date,checkout_date,special_requests,total_stays,previous_cancellations,status
0,5,101,Prakash Saini,prakash.mali23@st.niituniversity.in,9876543210,Executive Suite,1200.0,2024-01-10,2026-02-13,2026-02-16,Late checkout requested,2,1,Confirmed



--- All Bookings in Database ---
Total bookings found: 101


,booking_id,customer_id,name,email,phone,room_type,booking_price,booking_date,checkin_date,checkout_date,special_requests,total_stays,previous_cancellations,status
0,5,101,Prakash Saini,prakash.mali23@st.niituniversity.in,9876543210,Executive Suite,1200.00,2024-01-10,2026-02-13,2026-02-16,Late checkout requested,2,1,Confirmed
1,6,102,Laura Scott,ujones@example.net,(583)590-0205x8970,Standard,1865.99,2025-04-25,2026-03-06,2026-03-11,None,12,2,Confirmed
2,7,103,Mark Orozco,eileensmith@example.com,571-483-0400x29795,Suite,489.67,2025-05-31,2026-03-16,2026-03-23,None,17,0,Confirmed
3,8,104,Kyle Edwards,gregory32@example.com,(779)992-1040x983,Presidential,521.99,2025-03-04,2026-03-02,2026-03-03,Within play plan name culture nature might.,20,5,Cancelled
4,9,105,Robin Navarro,phamjohn@example.net,552.707.9318,Deluxe,681.47,2025-12-09,2026-03-16,2026-03-20,None,11,5,Pending
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,101,197,Jordan Johnson,manuel99@example.net,001-551-833-6402x92604,Deluxe,1498.93,2025-08-16,2026-02-22,2026-02-25,None,20,3,Confirmed
97,102,198,Diana Murphy,rberger@example.com,203-371-3111,Deluxe,1603.68,2025-10-04,2026-02-12,2026-02-14,None,8,3,Confirmed
98,103,199,Jay Mcgee,agillespie@example.org,227-551-5378x9923,Standard,934.85,2025-05-07,2026-03-22,2026-03-23,None,3,1,Cancelled
99,104,200,Andre Rose,fmckinney@example.org,656.708.9008,Standard,1985.43,2026-01-07,2026-02-18,2026-02-24,None,3,4,Confirmed
